In [1]:
import json
import copy
from pprint import pprint
import pandas as pd
from collections import OrderedDict
import os 

In [2]:
def json_unfold(df):
    result = []
    for i, row in df.iterrows():
        colums = row.index.values
        ds = [OrderedDict()]
        for colum in colums:
#             print(colum, row[colum], type(row[colum]))
            try:
                if isinstance(row[colum], list) or isinstance(
                        row[colum], dict):
                    json_o = row[colum]
                else:
                    json_o = json.loads(row[colum])
                if isinstance(json_o, list) and len(json_o) > 0 and isinstance(
                        json_o[0], dict):
                    bak = copy.deepcopy(ds[0])
                    for i, o in enumerate(json_o):
                        if i > 0:
                            d = copy.deepcopy(bak)
                            ds.append(d)
                        else:
                            d = ds[0]
                        for k, v in o.items():
                            d[f"{colum}-{k}"] = v
                else:
                    for k, v in json_o.items():
                        for d in ds:
                            d[f"{colum}-{k}"] = v
            except:
                for d in ds:
                    d[colum] = row[colum]
        for d in ds:
            result.append(d)

    return pd.DataFrame(result).dropna(axis=1, how='all')

In [3]:
os.chdir(r'F:\360MoveData\Users\Administrator\Desktop')
df = pd.read_excel('data1.xlsx')
display(df)
for i in range(2):
    df = json_unfold(df)
    display(df)

,序号,标注人员,标注结果
0,2,王小二,NaN
1,3,杨六,"{""tags"":[""96"",""f2931""],""username"":""楼满月"",""infos..."


,序号,标注人员,标注结果-tags,标注结果-username,标注结果-infos
0,2,王小二,NaN,NaN,NaN
1,3,杨六,"[96, f2931]",楼满月,"[{'地点': '湖北', '爱好': '足球'}]"


,序号,标注人员,标注结果-tags,标注结果-username,标注结果-infos-地点,标注结果-infos-爱好
0,2,王小二,NaN,NaN,NaN,NaN
1,3,杨六,"[96, f2931]",楼满月,湖北,足球
